# Data exploratory analysis 

### Load modules

In [2]:
# General
import os
import math
import re

# Data handling
import numpy as np
import pandas as pd

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Image handling
import cv2

## Load Data

### Load patient data

In [8]:
train_segmentation = pd.read_csv('data/train.csv')
train_segmentation.head(5)

,id,encoding
0,2f6ecfcdf,296084587 4 296115835 6 296115859 14 296147109...
1,8242609fa,96909968 56 96941265 60 96972563 64 97003861 6...
2,aaa6a05cc,30989109 59 31007591 64 31026074 68 31044556 7...
3,cb2d976f4,78144363 5 78179297 15 78214231 25 78249165 35...
4,b9a3865fc,61271840 4 61303134 13 61334428 22 61365722 30...


This csv file links the id of each sample to the ground truth segmentation in a run-length encoding

In [11]:
train_patient_data = pd.read_csv('data/HuBMAP-20-dataset_information.csv')
train_patient_data.head(5)

,image_file,width_pixels,height_pixels,anatomical_structures_segmention_file,glomerulus_segmentation_file,patient_number,race,ethnicity,sex,age,weight_kilograms,height_centimeters,bmi_kg/m^2,laterality,percent_cortex,percent_medulla
0,aa05346ff.tiff,47340,30720,aa05346ff-anatomical-structure.json,aa05346ff.json,67347,White,Not Hispanic or Latino,Female,58,59.0,160.0,23.0,Right,80,20
1,afa5e8098.tiff,43780,36800,afa5e8098-anatomical-structure.json,afa5e8098.json,67347,White,Not Hispanic or Latino,Female,58,59.0,160.0,23.0,Right,55,45
2,54f2eec69.tiff,22240,30440,54f2eec69-anatomical-structure.json,54f2eec69.json,67548,Black or African American,Not Hispanic or Latino,Male,58,79.9,190.5,22.0,Right,75,25
3,d488c759a.tiff,29020,46660,d488c759a-anatomical-structure.json,d488c759a.json,68138,White,Not Hispanic or Latino,Female,66,81.5,158.8,32.2,Left,100,0
4,1e2425f28.tiff,32220,26780,1e2425f28-anatomical-structure.json,1e2425f28.json,63921,White,Not Hispanic or Latino,Male,48,131.5,193.0,35.3,Right,65,35


This dataframe holds infomration about the size of the image as well as data from the patient like heigth, weight, sex, etc. There is some redundant information like the name of the files, that can be derived from the sample id. I will merge this dataframe and train_segmentation. 

In [13]:
# Add a column for the sample id in train_patient_data
train_patient_data['sample_id'] = train_patient_data.image_file.apply(lambda x: x.split('.')[0])

In [23]:
# Merge train_segmentation and train_patient_data using the sample id columns
train = pd.merge(train_segmentation, train_patient_data, left_on='id', right_on='sample_id')

# Remove non desired columns
train = train.drop(['id', 'image_file', 'anatomical_structures_segmention_file',
            'glomerulus_segmentation_file', 'patient_number'], axis=1)

# format column names
train.columns = ['encoding', 'width', 'height', 'race', 'ethnicity', 'sex',
       'age', 'weight', 'height', 'bmi', 'laterality', 'percent_cortex',
                 'percent_medulla', 'sample_id']

### Load images

In [ ]:
def load_image